# Classification_of_Consumer_Complaints(Japanese) by Llama3-70B 20240504

## 1. ライブラリをインポート (Import library)

In [ ]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!pip show langgraph

Name: langgraph
Version: 0.0.44
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core
Required-by: 


In [ ]:
!pip -q install langchain tiktoken  python-dotenv

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [ ]:
### Tracing (optional)
import os
from pprint import pprint

# os.environ['LANGCHAIN TRACING V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN API KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

## 2. モデルをセット (Set the model throught API)

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192",temperature=0)

In [ ]:
#Categorize complaints
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Categorizer Agent You are a master at understanding what product do customers complaint of in the complaints? You can choose only one product.

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the complaints provided and categorize into one of the following categories:
        Mortgage\
        Checking or savings account \
        Student loan\
        Money transfer, virtual currency, or money service\
        Bank account or service\
        Consumer Loan\

            Output a single cetgory only from the types ( 'Mortgage', 'Checking or savings account', 'Student loan', 'Money transfer, virtual currency, or money service',\
       'Bank account or service', 'Consumer Loan'. Your answer should be one of the name of product in the list only. Do not include other words. \) \
            eg:
            'Mortgage' \

    EMAIL CONTENT:\n\n {email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["email"],
)

email_category_generator = prompt | model | StrOutputParser()

morg="Greetree ( changed name to Ditech servicing ) calls me all hours of the night. A representative name XXXX calls me. I told them that the only number the are allowed to contact me on is XXXX. My mother has said they also call on their number which is a separate line from my own line. Even after telling them this for months and years they still abuse this. As I have said before that the only line that I can be reach is on XXXX. I work from home and that is the only number that they can contact me regarding any information about the mortgage. I do n't know how clear I can be on discussing this with them regarding XXXX constant harassment. I am also considering legal action."

result = email_category_generator.invoke({"email": morg})

print(result)

Mortgage


In [ ]:
morg

"Greetree ( changed name to Ditech servicing ) calls me all hours of the night. A representative name XXXX calls me. I told them that the only number the are allowed to contact me on is XXXX. My mother has said they also call on their number which is a separate line from my own line. Even after telling them this for months and years they still abuse this. As I have said before that the only line that I can be reach is on XXXX. I work from home and that is the only number that they can contact me regarding any information about the mortgage. I do n't know how clear I can be on discussing this with them regarding XXXX constant harassment. I am also considering legal action."

## 3. dataをインポート (load data)


In [ ]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report

In [ ]:
X=pd.read_csv("CC_20k_20230114.csv")
X

,Product,Issue,text,Complaint ID,Ja
0,Checking or savings account,Managing an account,XX/XX/2022. I began noticing charges to my deb...,6301154,xx/xx/2022。私は自分のデビットカードへの料金とオンラインウェルズファーゴアカウント...
1,Checking or savings account,Managing an account,I have a joint account with my Daughter. She g...,6280688,私は娘と共同アカウントを持っています。彼女はxxxx xxxxでxxxxに行き、私はxxxx...
2,Mortgage,Trouble during payment process,I made a principal payment of {$9900.00} on XX...,6257306,xx/xx/22で{$ 9900.00}の主要な支払いを行いました。米国銀行は、XXXX X...
3,Checking or savings account,Managing an account,I created a Savings account XX/XX/2022 with Al...,6246992,Ally Bankで普通預金口座XX/XX/2022を作成し、{$ 400.00}のデポジッ...
4,Mortgage,Trouble during payment process,Loan Number XXXX Reference : Mortgage XXXXXXXX...,6209018,ローン番号xxxxリファレンス：Mortgage xxxxxxx xxxx xxxx xxx...
...,...,...,...,...,...
199390,Bank account or service,"Account opening, closing, or management",Near the end of XXXX 2016 I opened a Citigold ...,2138969,XXXX 2016の終わり近くに、プロモーションコードXXXXでCitGold当座預金口座を...
199391,Student loan,Dealing with my lender or servicer,My loans have an extraordinarily high interest...,2302907,私のローンには非常に高い金利があり、私はそれらを借り換えることができません。
199392,Mortgage,"Loan modification,collection,foreclosure",Caliber Home Loans ( CHL ) sent confirmation i...,2302852,キャリバーホームローン（CHL）は、2016年のXXXXで確認を送信し、ローンの変更が承認さ...
199393,Consumer Loan,Managing the loan or lease,On XX/XX/XXXX I purchased a XXXX XXXX XXXX XXX...,2282405,xx/xx/xxxxでは、xxxx xxxx xxxx xxxxを購入しました。オハイオ州ド...


In [ ]:
from sklearn.preprocessing import LabelEncoder

lab = LabelEncoder()
lab = lab.fit(X["Product"])
X["label"] = lab.transform(X["Product"])
X.head()

,Product,Issue,text,Complaint ID,Ja,label
0,Checking or savings account,Managing an account,XX/XX/2022. I began noticing charges to my deb...,6301154,xx/xx/2022。私は自分のデビットカードへの料金とオンラインウェルズファーゴアカウント...,1
1,Checking or savings account,Managing an account,I have a joint account with my Daughter. She g...,6280688,私は娘と共同アカウントを持っています。彼女はxxxx xxxxでxxxxに行き、私はxxxx...,1
2,Mortgage,Trouble during payment process,I made a principal payment of {$9900.00} on XX...,6257306,xx/xx/22で{$ 9900.00}の主要な支払いを行いました。米国銀行は、XXXX X...,4
3,Checking or savings account,Managing an account,I created a Savings account XX/XX/2022 with Al...,6246992,Ally Bankで普通預金口座XX/XX/2022を作成し、{$ 400.00}のデポジッ...,1
4,Mortgage,Trouble during payment process,Loan Number XXXX Reference : Mortgage XXXXXXXX...,6209018,ローン番号xxxxリファレンス：Mortgage xxxxxxx xxxx xxxx xxx...,4


In [ ]:
Xp=X[0:5000]
Xp["Product"].value_counts()

Product
Checking or savings account                           2000
Mortgage                                              1660
Student loan                                           628
Money transfer, virtual currency, or money service     428
Bank account or service                                192
Consumer Loan                                           92
Name: count, dtype: int64

In [ ]:
X=X[5000:]

In [ ]:
X["Product"].value_counts()

Product
Mortgage                                              88101
Checking or savings account                           52372
Student loan                                          26787
Money transfer, virtual currency, or money service    13184
Bank account or service                                9546
Consumer Loan                                          4405
Name: count, dtype: int64

In [ ]:
Xt=X[["Ja"]]
Xt

,Ja
5000,XXXXでは、一部の債務を統合するために、信用組合と信用枠を申請しました。彼らの検索は、私が...
5001,XX/XX/XXXX M＆T Bankが私に電話をかけ、私のアカウントが{$ 2000.00...
5002,xx/xx/xxxxでは、オンラインで時計を購入し、xxxxを使用して売り手に{$ 4500...
5003,タイトルはまさに述べられたとおりです。\n\nXXXXに戻ってTD銀行は、非行のために私の口...
5004,私はウェルズ・ファーゴとのローンを借り換えました。そもそも悪夢でしたが、最終的にXXXXで最...
...,...
199390,XXXX 2016の終わり近くに、プロモーションコードXXXXでCitGold当座預金口座を...
199391,私のローンには非常に高い金利があり、私はそれらを借り換えることができません。
199392,キャリバーホームローン（CHL）は、2016年のXXXXで確認を送信し、ローンの変更が承認さ...
199393,xx/xx/xxxxでは、xxxx xxxx xxxx xxxxを購入しました。オハイオ州ド...


In [ ]:
yl=X[["label"]]
yl

,label
5000,2
5001,1
5002,3
5003,1
5004,4
...,...
199390,0
199391,5
199392,4
199393,2


In [ ]:
max(yl["label"])

5

## 4. 訓練データと検証データを作成　(Prepare training and test data)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(Xt, yl, stratify=yl, train_size=0.013, test_size=0.013, random_state=100)
len(X_train), len(y_train), len(X_valid), len(y_valid)

(2527, 2527, 2528, 2528)

In [ ]:
X_valid

,Ja
85871,私は最近、xxxx xxxx xxxxからseterusに販売されました。XXXX XXXX...
113030,2番目の住宅ローンサービサーPlanet Home Lending Acc＃xxxxは、「そ...
57238,xxxx xxxxは彼女の家に住んでいる年配の女性です。彼女は必死に家を維持し、ローンを復活...
130633,私の住宅ローンはXXXX 2015でNationStar住宅ローンに譲渡されました。この会社...
26169,Ocwen Companyは、会社のさまざまな分野で指示を伝える大きな問題を抱えています。な...
...,...
159445,この苦情の理由は、このXXXXカレッジローンの利息と元本に割り当てられた金額が間違っていると...
157764,私たちは現在、住宅ローンで滞納しており、XXXX/XXXX/XXXXの販売日を取得しています...
25321,これは、この会社に対して提出された最大3つの以前の苦情です。私は12か月の返済計画を立ててい...
120677,私のローンサービサーであるキャンパスパートナーは、返済通知が一貫して遅刻しています。最後のx...


In [ ]:
y_valid

,label
85871,4
113030,4
57238,4
130633,4
26169,4
...,...
159445,5
157764,4
25321,4
120677,5


In [ ]:
y_valid.nunique()

label    6
dtype: int64

In [ ]:
data = pd.concat([y_valid, X_valid], axis=1)
data

,label,Ja
85871,4,私は最近、xxxx xxxx xxxxからseterusに販売されました。XXXX XXXX...
113030,4,2番目の住宅ローンサービサーPlanet Home Lending Acc＃xxxxは、「そ...
57238,4,xxxx xxxxは彼女の家に住んでいる年配の女性です。彼女は必死に家を維持し、ローンを復活...
130633,4,私の住宅ローンはXXXX 2015でNationStar住宅ローンに譲渡されました。この会社...
26169,4,Ocwen Companyは、会社のさまざまな分野で指示を伝える大きな問題を抱えています。な...
...,...,...
159445,5,この苦情の理由は、このXXXXカレッジローンの利息と元本に割り当てられた金額が間違っていると...
157764,4,私たちは現在、住宅ローンで滞納しており、XXXX/XXXX/XXXXの販売日を取得しています...
25321,4,これは、この会社に対して提出された最大3つの以前の苦情です。私は12か月の返済計画を立ててい...
120677,5,私のローンサービサーであるキャンパスパートナーは、返済通知が一貫して遅刻しています。最後のx...


## 5. Llama3-70Bによりクレームを判別 (Classify customer complaints by Llama3-70B)

In [ ]:
%%time

result=[]
n=0
m=500

for i in range(n,m):

  out = email_category_generator.invoke({"email": data.iloc[i,1]})
  result.append(out)

result

CPU times: user 20.3 s, sys: 2.16 s, total: 22.5 s
Wall time: 46min 17s


['Consumer Loan',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Money transfer, virtual currency, or money service',
 'Checking or savings account',
 'Money transfer, virtual currency, or money service',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Student loan',
 'Checking or savings account',
 'Mortgage',
 'Student loan',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Consumer Loan',
 'Mortgage',
 'Student loan',
 'Consumer Loan',
 'Consumer Loan',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Checking or savings account',


In [ ]:
range(n,m)

range(0, 500)

In [ ]:
 list(lab.inverse_transform([0, 1, 2, 3, 4, 5]))

['Bank account or service',
 'Checking or savings account',
 'Consumer Loan',
 'Money transfer, virtual currency, or money service',
 'Mortgage',
 'Student loan']

In [ ]:
s=result.copy()
for i in range(0,m-n):
    if "Bank account or service" in s[i]:
      s[i]=0
    elif "Checking or savings account" in s[i]:
      s[i]=1
    elif "Consumer Loan" in s[i]:
      s[i]=2
    elif "Money transfer, virtual currency, or money service" in s[i]:
      s[i]=3
    elif "Mortgage" in s[i]:
      s[i]=4
    elif "Student loan" in s[i]:
      s[i]=5
    else:
      s[i]=6
#s

## 6. 精度検証　(Evaluate the model)

In [ ]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d3559bfa140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d3559bf9630>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [ ]:
label=list(y_valid['label'])[n:m]

print("result:", s)
print("label:", label)

result: [2, 4, 4, 4, 4, 4, 1, 4, 1, 4, 3, 1, 3, 1, 4, 4, 4, 4, 4, 1, 5, 1, 4, 5, 4, 4, 4, 4, 2, 4, 5, 2, 2, 1, 4, 1, 4, 4, 1, 4, 4, 4, 1, 4, 4, 1, 4, 1, 1, 4, 4, 1, 2, 4, 4, 4, 4, 5, 4, 4, 4, 0, 3, 5, 1, 1, 4, 0, 4, 4, 1, 3, 3, 1, 2, 1, 1, 1, 1, 0, 4, 4, 1, 4, 1, 4, 4, 1, 4, 1, 4, 4, 4, 4, 4, 5, 1, 4, 1, 5, 4, 4, 2, 4, 1, 4, 4, 1, 1, 4, 4, 4, 1, 4, 0, 5, 0, 4, 4, 4, 4, 5, 1, 4, 2, 0, 5, 1, 5, 3, 4, 4, 5, 1, 4, 2, 5, 1, 4, 1, 4, 4, 5, 4, 5, 4, 5, 1, 0, 6, 4, 4, 1, 4, 4, 4, 3, 1, 2, 4, 0, 1, 1, 1, 4, 5, 4, 3, 1, 1, 1, 4, 4, 3, 4, 4, 1, 4, 4, 1, 3, 5, 4, 5, 1, 4, 1, 4, 5, 1, 1, 1, 4, 4, 4, 3, 4, 0, 3, 1, 2, 4, 5, 3, 4, 5, 4, 4, 2, 4, 4, 4, 0, 4, 4, 5, 5, 4, 1, 4, 4, 1, 4, 4, 1, 1, 4, 3, 4, 2, 1, 4, 1, 4, 4, 1, 1, 4, 4, 4, 1, 1, 4, 1, 3, 1, 1, 4, 4, 5, 4, 1, 4, 4, 1, 1, 4, 3, 4, 2, 4, 1, 5, 2, 0, 5, 1, 4, 4, 4, 1, 4, 1, 4, 4, 1, 4, 4, 2, 4, 1, 4, 5, 5, 5, 1, 1, 1, 4, 1, 1, 1, 4, 4, 3, 4, 4, 4, 1, 2, 1, 4, 5, 1, 4, 2, 4, 5, 4, 4, 1, 1, 1, 4, 5, 4, 1, 4, 4, 4, 2, 4, 4, 2, 4, 1, 1, 1, 4, 5, 1

分布を確認 (Check the distribution of data)

In [ ]:
sdf=pd.Series(s).value_counts()
sdf.sort_index()

0     19
1    145
2     30
3     24
4    224
5     57
6      1
Name: count, dtype: int64

In [ ]:
labeldf=pd.Series(label).value_counts()
labeldf.sort_index()

0     26
1    119
2     16
3     38
4    232
5     69
Name: count, dtype: int64

In [ ]:
print('accuracy score: {0:0.4f}'.format(accuracy_score(label, s)))


accuracy score: 0.8280


In [ ]:
print(classification_report(label, s, digits=4))

              precision    recall  f1-score   support

           0     0.0526    0.0385    0.0444        26
           1     0.7172    0.8739    0.7879       119
           2     0.4333    0.8125    0.5652        16
           3     0.9583    0.6053    0.7419        38
           4     0.9643    0.9310    0.9474       232
           5     1.0000    0.8261    0.9048        69
           6     0.0000    0.0000    0.0000         0

    accuracy                         0.8280       500
   macro avg     0.5894    0.5839    0.5702       500
weighted avg     0.8456    0.8280    0.8287       500



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### accuracy score
Test data : 500

- Llama3-70B : accuracy 0.886, time 25m52s      


Test data : 500 (Japanese)

- Llama3-70B : accuracy 0.828, time 46m17s    
